In [12]:
import numpy as np
import pandas as pd
import cvxopt

In [4]:
sample = pd.read_csv("/Users/jialanma/Downloads/svm-test-datasets/2d_binary.txt",header=None)
sample

,0,1,2
0,10.028628,4.958046,1.0
1,8.915562,7.350755,1.0
2,6.661766,4.406966,1.0
3,8.964600,6.139508,1.0
4,9.288912,2.702496,1.0
...,...,...,...
195,6.164281,2.282623,1.0
196,3.747496,10.849596,-1.0
197,9.032419,6.817468,1.0
198,6.114723,2.799260,1.0


In [15]:
x = np.array(sample.iloc[:,0:2]) 
y = np.array(sample.iloc[:,2])

In [26]:
length = len(y)
length

200

In [116]:
# Split in half for training and testing data
x_train = x[:length//2]
x_test = x[length//2:]

y_train = y[:length//2]
y_test = y[length//2:]

In [117]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((100, 2), (100, 2), (100,), (100,))

In [49]:
# Build the parameters for QP solver
# H is a matrix of size (n,n) such that Hi,j=y(i)y(j)<x(i)x(j)>
row, col = x_train.shape[0], x_train.shape[0]
H = np.zeros([row,col])
for i in range(row):
    for j in range(col):
        H[i,j] = y_train[i]*y_train[j]*x_train[i,:].dot(x_train[j,:])


In [68]:
# q is a nx1 vector with -1
q = -np.ones(row)
q

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1.])

In [69]:
# G is an identity matrix with -1's
G = -np.eye(row)
G

array([[-1., -0., -0., ..., -0., -0., -0.],
       [-0., -1., -0., ..., -0., -0., -0.],
       [-0., -0., -1., ..., -0., -0., -0.],
       ...,
       [-0., -0., -0., ..., -1., -0., -0.],
       [-0., -0., -0., ..., -0., -1., -0.],
       [-0., -0., -0., ..., -0., -0., -1.]])

In [60]:
# h is a nx1 vector of 0's
h = np.zeros(row)
h

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [75]:
# A is a nx1 vector of label y's
A = y_train.reshape(1,-1)
A.shape

(1, 100)

In [64]:
# b is a scalar, zero
b = 0

In [65]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

In [79]:
#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(np.zeros(1))

In [80]:
#Setting solver parameters (change default to decrease tolerance) 
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

In [81]:
#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

In [119]:
train_len = y_train.shape[0]
train_len

100

In [123]:
alphas = alphas.reshape(1,-1)
alphas.shape

(1, 100)

In [121]:
y_train.shape

(100,)

In [122]:
y_matrix = np.eye(train_len)
for i in range(train_len):
    y_matrix[i,i] = y_train[i]
y_matrix

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [124]:
# solve for w's and b's
lambda_y = np.dot(alphas,y_matrix)
weights = np.dot(lambda_y,x_train)
weights

array([[ 0.64362514, -1.00000044]])

In [146]:
#Selecting the set of indices S corresponding to non zero parameters
S = (alphas > 1e-4).flatten()
S

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False, False, False, False,
       False])

In [154]:
#Computing b
b = y_train[S].reshape(3,1) - np.dot(x_train[S], weights.reshape(-1,1))

In [155]:
b

array([[2.77022397],
       [2.77022397],
       [2.77022397]])

In [156]:
# Column mean
b_final = b.mean(0)
b_final

array([2.77022397])

In [157]:
#Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', weights.flatten())
print('b = ', b_final)

Alphas =  [0.01659302 0.69053409 0.7071271 ]
w =  [ 0.64362514 -1.00000044]
b =  [2.77022397]


In [158]:
weights.T.shape

(2, 1)

In [162]:
# Test 
y_predict = np.dot(x_test,weights.T) + b_final
for i in range(len(y_predict)):
    if y_predict[i] > 0:
        y_predict[i] = 1
    else:  # <= 0
        y_predict[i] = -1

In [163]:
y_predict[0]

array([-1.])

In [165]:
y_test[0] == y_predict[0]

array([ True])

In [166]:
count = 0
for i in range(len(y_predict)):
    if y_test[i] == y_predict[i]:
        count += 1
count 

100

In [263]:
X = np.array([[3,4],[1,4],[2,3],[6,-1],[7,-1],[5,-3],[2,4]] )
y = np.array([-1,-1, -1, 1, 1 , 1, 1 ]).reshape(-1,1) * 1.

In [264]:
X.shape,y.shape

((7, 2), (7, 1))

In [265]:
m,n = X.shape

In [266]:
X_dash = y * X #yixi

In [281]:
H = np.dot(X_dash,X_dash.T) *1. #yixiyjxj

In [268]:
q = -np.ones((m,1))

In [269]:
A = y.reshape(1,-1)
A.shape

(1, 7)

In [270]:
b = np.zeros(1)

In [271]:
top_G = -np.eye(m)
bot_G = np.eye(m)
G_prime = np.concatenate((top_G,bot_G))
G_prime

array([[-1., -0., -0., -0., -0., -0., -0.],
       [-0., -1., -0., -0., -0., -0., -0.],
       [-0., -0., -1., -0., -0., -0., -0.],
       [-0., -0., -0., -1., -0., -0., -0.],
       [-0., -0., -0., -0., -1., -0., -0.],
       [-0., -0., -0., -0., -0., -1., -0.],
       [-0., -0., -0., -0., -0., -0., -1.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [304]:
h = np.zeros([2*m])
for i in range(m,2*m):
    h[i] = C

In [305]:
h

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 10., 10., 10., 10., 10.,
       10.])

In [306]:
#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G_prime)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(b)

In [307]:
#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

In [310]:
alphas = alphas.reshape(1,-1)
alphas.shape

(1, 7)

In [311]:
id_m = np.eye(m)
for i in range(m):
    id_m[i,i] = y[i]
id_m

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [312]:
id_m.shape

(7, 7)

In [313]:
alpha_times_y = np.dot(alphas,id_m)
w_vector = np.dot(alpha_times_y,X)
w_vector

array([[ 0.25, -0.25]])

In [314]:
S = (alphas > 1e-4).flatten() # alphas considered non-zero
S

array([ True, False,  True,  True, False, False,  True])

In [321]:
b = y[S] - np.dot(X[S],w_vector.reshape(-1,1))
b

array([[-0.75],
       [-0.75],
       [-0.75],
       [ 1.5 ]])

In [282]:
C = 10
m,n = X.shape
y = y.reshape(-1,1) * 1.
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

#Converting into cvxopt format - as previously
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

alphas

array([[5.00000000e+00],
       [1.52461667e-12],
       [6.31250000e+00],
       [1.31250000e+00],
       [2.36607341e-12],
       [1.32909858e-11],
       [1.00000000e+01]])

In [290]:
np.hstack((np.zeros(m), np.ones(m) * C))

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0., 10., 10., 10., 10., 10., 10.,
       10.])

#### Soft-margin SVM with cancer data

In [662]:
cancer_dat = pd.read_csv("/Users/jialanma/Downloads/cancer.csv")
cancer_dat

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,M,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,M,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,M,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,M,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [663]:
X = np.array(cancer_dat.iloc[:,2:-1])
diagnosis = np.array(cancer_dat.iloc[:,1])

In [664]:
m,n = X.shape
m,n

(569, 30)

In [665]:
y = np.zeros(m)
for i in range(m):
    if diagnosis[i] == "M":
        y[i] = 1
    else:
        y[i] = -1

In [666]:
# Split data for training and testing
X_train, X_test = X[:300],X[300:]
y_train, y_test = y[:300],y[300:]

In [667]:
y_train = y_train.reshape(-1,1)*1.
y_train.shape

(300, 1)

In [668]:
m_train,n_train = X_train.shape

In [354]:
y_train * X_train

array([[ 1.799e+01,  1.038e+01,  1.228e+02, ...,  2.654e-01,  4.601e-01,
         1.189e-01],
       [ 2.057e+01,  1.777e+01,  1.329e+02, ...,  1.860e-01,  2.750e-01,
         8.902e-02],
       [ 1.969e+01,  2.125e+01,  1.300e+02, ...,  2.430e-01,  3.613e-01,
         8.758e-02],
       ...,
       [ 1.176e+01,  1.814e+01,  7.500e+01, ...,  7.160e-02,  1.978e-01,
         6.915e-02],
       [-1.426e+01, -1.817e+01, -9.122e+01, ..., -7.530e-02, -2.636e-01,
        -7.676e-02],
       [-1.051e+01, -2.309e+01, -6.685e+01, ..., -3.125e-02, -2.227e-01,
        -6.777e-02]])

In [358]:
C = 10

In [385]:
def transform_params(X_train,y_train,C):
    m = X_train.shape[0]
    X_times_y = y_train * X_train
    H = np.dot(X_times_y,X_times_y.T)
    q = -np.ones((m,1))
    G = np.vstack((-np.eye(m),np.eye(m)))
    h = np.hstack((np.zeros(m),C*np.ones(m)))
    A = y_train.reshape(1,-1)
    b = np.zeros(1)
    
    return H,q,G,h,A,b

In [386]:
H,q,G,h,A,b = transform_params(X_train,y_train,C)

In [387]:
#Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(q)
G = cvxopt_matrix(G)
h = cvxopt_matrix(h)
A = cvxopt_matrix(A)
b = cvxopt_matrix(b)

In [388]:
#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

In [397]:
alphas = alphas.reshape(1,-1)
alphas.shape

(1, 300)

In [399]:
y_id = np.eye(m_train)
for i in range(m_train):
    y_id[i,i] = y_train[i]
y_id

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [414]:
alpha_times_y = np.dot(alphas,y_id)
weights = np.dot(alpha_times_y,X_train)
weights = weights.reshape(-1,1)
weights.shape

(30, 1)

In [433]:
b = y_train[S] - np.dot(X_train[S],weights)
b_ave = b.mean()

In [417]:
m_test,n_test = X_test.shape

In [434]:
y_pred = np.zeros(m_test)
for i in range(m_test):
    sign = np.dot(weights.T,X_test[i,:].reshape(-1,1))+b_ave
    if sign > 0:
        y_pred[i] = 1
    else:
        y_pred[i] = -1

In [435]:
count = 0
for i in range(m_test):
    if y_pred[i] != y_test[i]:
        count += 1
count

12

In [437]:
(m_test-count)/m_test

0.9553903345724907

In [444]:
# Compare with sklearn
from sklearn.svm import SVC

In [446]:
clf = SVC(C = 10, kernel = 'linear')
clf.fit(X_train, y_train.ravel()) 

SVC(C=10, kernel='linear')

In [447]:
print('w = ',clf.coef_)
print('b = ',clf.intercept_)

w =  [[-1.12594155e+00  4.76769635e-02 -2.89523520e-01 -6.21557917e-05
   9.65460384e-01  2.12823199e-01  1.26726334e+00  1.01186332e+00
   4.96098433e-01  1.23220466e-01 -9.67199176e-01 -1.27065905e+00
   7.29355793e-01  1.06257769e-01  3.03220035e-01 -2.42455491e-01
   5.47276265e-01  3.74789009e-01  9.53261239e-02 -9.42501311e-02
  -2.55660698e+00  4.50266270e-01  4.20956201e-01  3.46769280e-02
   1.27765434e+00 -1.67402131e-01  3.76449466e+00  2.41547170e+00
   1.39789133e+00  1.00864183e-01]]
b =  [-6.97261855]


In [462]:
y_pred_sk = clf.predict(X_test)

In [459]:
count_sk_g = 0
count_sk_qp = 0
for i in range(m_test):
    if y_pred_sk[i] != y_test[i]:
        count_sk_g += 1
    if y_pred_sk[i] != y_pred[i]:
        count_sk_qp += 1

In [460]:
count_sk_g,count_sk_qp

(15, 7)

In [461]:
(m_test-count_sk_g)/m_test

0.9442379182156134

#### Sequential Minimal Optimization

In [518]:
import random

In [669]:
def kernel(x1,x2):
    # dot product as kernel function
    k12 = np.dot(x1.reshape(1,-1),x2.reshape(-1,1))
    return k12

In [670]:
def calculate_E(alpha,y_train,X_train,b,x_i,y_i):
    f_xi = 0
    for j in range(len(alpha)):
        xj = X_train[j,:]
        xi = x_i
        kji = kernel(xj,xi)
        f_xi += alpha[j]*y_train[j]*kji
    
    f_xi += b 
    Ei = f_xi - y_i
    return Ei

In [671]:
def calculate_LH(C,y_i,y_j,alpha_i,alpha_j):
    if y_i != y_j:
        L = max(0, alpha_j-alpha_i)
        H = min(C, C+alpha_j-alpha_i)
    else:
        L = max(0, alpha_i+alpha_j-C)
        H = min(C, alpha_i+alpha_j)
        
    return L,H

In [672]:
def calculate_eita(x_i,x_j):
    eita = 2*kernel(x_i,x_j)-kernel(x_i,x_i)-kernel(x_j,x_j)
    return eita

In [673]:
def calculate_a2(a2_old,Ei,Ej,eita,y_j,L,H):
    a2_new = a2_old - y_j*(Ei-Ej)/eita
    
    if a2_new > H:
        a2_new = H
    elif a2_new < L:
        a2_new = L
    
    return a2_new

In [751]:
m_train,n_train = X_train.shape

In [847]:
# Initialization: all alphas and b are zero
alpha_old,alpha_new = np.zeros((m_train,1)),np.zeros((m_train,1))
b = 0

In [848]:
C = 10 # regularization param
epsilon = 1e-3 # numerical tolerance for KKT conditions
max_passes = 1 # maximum numbe of passes to iterate over alpha's without changing

In [849]:
passes = 0
while passes < max_passes:
    num_changed_alphas = 0
    for i in range(m_train):
        a_i = alpha_new[i]
        x_i = X_train[i,:]
        y_i = y_train[i]
        Ei = calculate_E(alpha_new,y_train,X_train,b,x_i,y_i)
        if (y_i*Ei < -epsilon and a_i < C) or (y_i*Ei > epsilon and a_i > 0): # KKT condition for unbound
            # Choose a random j that is not equal to i
            j = random.randint(0,m_train-1)
            while j == i:
                j = random.randint(0,m_train-1)
            a_j = alpha_new[j]
            x_j = X_train[j,:]
            y_j = y_train[j]
            Ej = calculate_E(alpha_new,y_train,X_train,b,x_j,y_j)
            
            alpha_old[i] = alpha_new[i]
            alpha_old[j] = alpha_new[j]
            
            L,H = calculate_LH(C,y_i,y_j,a_i,a_j)
            if L == H:
                continue
                
            eita = calculate_eita(x_i,x_j)
            if eita >= 0:
                continue
  
            a_j_old = a_j
            a_j_new = calculate_a2(a_j_old,Ei,Ej,eita,y_j,L,H)
            
            #print(a_j_new,a_j_old)
            if abs(a_j_new-a_j_old) < 1e-4:
                continue
            
                
            a_i_old = a_i
            a_i_new = a_i_old+y_i*y_j*(a_j_old-a_j_new)  ##### Update alpha_old
            alpha_new[i] = a_i_new
            
            b1 = b-Ei-y_i*(a_i_new-a_i_old)*kernel(x_i,x_i)-y_j*(a_j_new-a_j_old)*kernel(x_i,x_j) # +-? 
            b2 = b-Ej-y_i*(a_i_new-a_i_old)*kernel(x_i,x_j)-y_j*(a_j_new-a_j_old)*kernel(x_j,x_j)
            if (a_i_new > 0 and a_i_new < C):
                b = b1
            elif (a_j_new > 0 and a_j_new < C):
                b = b2
            else:
                b = (b1+b2)/2
            
            alpha_new[j] = a_j_new
            alpha_new[i] = a_i_new
            num_changed_alphas += 1
            
    if num_changed_alphas == 0:
        passes += 1
    else:
        passes = 0
            

In [850]:
alpha_new = alpha_new.reshape(1,-1)
alpha_new.shape

(1, 300)

In [851]:
alpha_new[alpha_new>1e-4]

array([0.00034926, 0.0001103 , 0.00011202, 0.00049683, 0.00140324,
       0.00089966, 0.00032611, 0.00010077, 0.00032611, 0.00037444,
       0.00803534, 0.00852752, 0.0001961 , 0.00043028, 0.00023526,
       0.0001961 , 0.0001361 , 0.00010769, 0.0001103 , 0.00034926,
       0.00010077, 0.00012757, 0.00108241, 0.0014102 ])

In [852]:
y_id = np.eye(m_train)
for i in range(m_train):
    y_id[i,i] = y_train[i]
y_id.shape

(300, 300)

In [853]:
alpha_times_y = np.dot(alpha_new,y_id)
weights = np.dot(alpha_times_y,X_train)
weights = weights.reshape(-1,1)
weights.shape

(30, 1)

In [854]:
tmp=b
tmp

array([[-132.53625226]])

In [855]:
b = y_train[S] - np.dot(X_train[S],weights)
b_ave = b.mean()

In [856]:
b_ave

-8.674400951697805

In [857]:
m_test,n_test = X_test.shape

In [858]:
y_pred = np.zeros(m_test)
for i in range(m_test):
    sign = np.dot(weights.T,X_test[i,:].reshape(-1,1))+b_ave
    if sign > 0:
        y_pred[i] = 1
    else:
        y_pred[i] = -1

In [859]:
count = 0
for i in range(m_test):
    if y_pred[i] != y_test[i]:
        count += 1
count

13

In [860]:
(m_test-count)/m_test

0.9516728624535316

In [846]:
(m_test-count)/m_test

0.8773234200743495